# 2018

In [1]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

# Origin-Destination Data

# Old Folder - 2018 Data 
- This is the LEHD data that we QCd before, that Anne re-ran for us
- All of this data can be seen looked at in the 'New-Runs' page
- This is not the new 2018 data

In [2]:
path = r'R:/DPOE/LEHD LODES/7.5/Source/OD_Data/Unzipped_RD'

In [3]:
# 2018 source data
csv_2018 = glob.glob(path + "/*2018.csv")
list_2018 = []

for filename in csv_2018:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_2018.append(df)

In [4]:
od_2018 = pd.concat(list_2018, axis=0, ignore_index=True)

In [5]:
od_2018 = od_2018.rename({'w_geocode': 'w_geoid', 'h_geocode': 'h_geoid'}, axis=1)
od_2018 = od_2018.drop(['createdate',], axis=1)

In [31]:
od_2018.dtypes

w_geoid    float64
h_geoid    float64
S000         int64
SA01         int64
SA02         int64
SA03         int64
SE01         int64
SE02         int64
SE03         int64
SI01         int64
SI02         int64
SI03         int64
dtype: object

In [6]:
old_od_2018 = od_2018 [od_2018['w_geoid'].astype(str).str.startswith('6073') | od_2018['h_geoid'].astype(str).str.startswith('6073')]

# New Folder - 2018 Data
- This is looking in the new folder  that has 2019 values, but we will be extracting 2018 values to compare.

In [7]:
path = r'R:/DPOE/LEHD LODES/7.5_2021_11Nov/Source/LODES_Download_2021-11-19-20-07-45/od'

csv_2018 = glob.glob(path + "/*2018.csv")
list_2018 = []

for filename in csv_2018:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_2018.append(df)

In [8]:
new_od_2018 = pd.concat(list_2018, axis=0, ignore_index=True)

In [9]:
new_od_2018 = new_od_2018.rename({'w_geocode': 'w_geoid', 'h_geocode': 'h_geoid'}, axis=1)
new_od_2018 = new_od_2018.drop(['createdate',], axis=1)

In [32]:
new_od_2018.dtypes

w_geoid    int64
h_geoid    int64
S000       int64
SA01       int64
SA02       int64
SA03       int64
SE01       int64
SE02       int64
SE03       int64
SI01       int64
SI02       int64
SI03       int64
dtype: object

In [10]:
new_od_2018_sub = new_od_2018[new_od_2018['w_geoid'].astype(str).str.startswith('6073') | new_od_2018['h_geoid'].astype(str).str.startswith('6073')]

In [11]:
new_od_2018_sub.equals(old_od_2018)

False

# Checking the Difference Between the two

In [2]:
old_od_2018

NameError: name 'old_od_2018' is not defined

In [13]:
new_od_2018_sub

,w_geoid,h_geoid,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03
123292,60730001002001,40190046411041,1,0,0,1,0,0,1,0,0,1
123293,60730001002002,40190040732033,1,0,0,1,1,0,0,0,0,1
123294,60730002021006,390450325002034,1,0,1,0,0,1,0,0,0,1
123295,60730002021008,310479685001032,1,0,1,0,0,0,1,0,0,1
123296,60730002023001,390490043004003,1,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
56937830,60971521001027,60730105024013,1,0,1,0,0,0,1,0,0,1
56937833,60971521001028,60730033052000,1,0,1,0,0,0,1,0,0,1
56939111,61110015031016,60730062001020,1,0,0,1,0,0,1,0,0,1
56939112,61110015031016,60730063001026,1,0,0,1,0,0,1,0,0,1


In [22]:
# Checking lengths of the data
print(len(old_od_2018))
print(len(new_od_2018_sub))

5685834
5699729


In [25]:
#Checking the number of rows more the new data has
len(new_od_2018_sub) - len(old_od_2018)

13895

In [27]:
# Checking the number of unique 'w_geoids'
print(old_od_2018['w_geoid'].nunique())
print(new_od_2018_sub['w_geoid'].nunique())

60759
60759


In [29]:
# Checking the number of unique 'h_geoids'
print(old_od_2018['h_geoid'].nunique())
print(new_od_2018_sub['h_geoid'].nunique())

154602
154673


In [30]:
# Checking the number of more unique h_geoids in new OD
new_od_2018_sub['h_geoid'].nunique() - old_od_2018['h_geoid'].nunique()

71

In [40]:
set(new_od_2018_sub['h_geoid']) - set(old_od_2018['h_geoid'])
h_geoids_in_new = {'h_geoid In New not Old':list(set(new_od_2018_sub['h_geoid']) - set(old_od_2018['h_geoid']))}

In [41]:
pd.DataFrame(data=h_geoids_in_new)

,h_geoid In New not Old
0,550250014032000
1,180390002001025
2,490532716001029
3,350479576001030
4,320310029024007
...,...
66,291319629006069
67,220550010022009
68,40134226402042
69,461030106001019


In [44]:
pd.DataFrame(data=h_geoids_in_new).to_csv('C:/Users/cra/Desktop/unique_h.csv', index=False) 

In [52]:
test = old_od_2018.head(10)
test

,w_geoid,h_geoid,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03
212905,6.073000e+13,4.019005e+13,1,0,0,1,0,0,1,0,0,1
212906,6.073000e+13,4.019004e+13,1,0,0,1,1,0,0,0,0,1
212907,6.073000e+13,3.104797e+14,1,0,1,0,0,0,1,0,0,1
212908,6.073000e+13,5.011950e+13,1,0,1,0,0,1,0,0,0,1
212909,6.073000e+13,4.013107e+13,1,0,1,0,0,1,0,0,0,1
212910,6.073000e+13,4.013061e+13,1,0,0,1,0,1,0,0,0,1
212911,6.073000e+13,3.203100e+14,1,1,0,0,0,1,0,0,0,1
212912,6.073000e+13,4.019005e+13,1,1,0,0,0,0,1,0,1,0
212913,6.073000e+13,4.714398e+14,1,0,1,0,0,0,1,0,1,0
212914,6.073000e+13,9.001074e+13,1,1,0,0,0,1,0,0,1,0


In [60]:
test['w_geoid']

212905    6.073000e+13
212906    6.073000e+13
212907    6.073000e+13
212908    6.073000e+13
212909    6.073000e+13
212910    6.073000e+13
212911    6.073000e+13
212912    6.073000e+13
212913    6.073000e+13
212914    6.073000e+13
Name: w_geoid, dtype: float64

In [63]:
old_data = old_od_2018.astype({'w_geoid': np.int64, 'h_geoid': np.int64})

In [108]:
old_data_w = [str(x) for x in list(old_data['w_geoid'])]

In [109]:
old_data_h = [str(x) for x in list(old_data['h_geoid'])]

In [110]:
old_data_trips = []
for i in range(len(old_data)):
    w = old_data_w[i]
    h = old_data_h[i]
    old_data_trips.append(w+'-'+h)

In [111]:
len(old_data_trips)

5685834

In [112]:
new_data_w = [str(x) for x in list(new_od_2018_sub['w_geoid'])]
new_data_h = [str(x) for x in list(new_od_2018_sub['h_geoid'])]

In [113]:
new_data_trips = []
for i in range(len(new_od_2018_sub)):
    w = new_data_w[i]
    h = new_data_h[i]
    new_data_trips.append(w+'-'+h)

In [1]:
len(new_data_trips)

NameError: name 'new_data_trips' is not defined

In [ ]:
# merged = old_data.merge(new_od_2018_sub, left_on=['w_geoid','h_geoid'],
#                                                   right_on=['w_geoid','h_geoid'], 
#                                                   how='right', suffixes=['_old','_new'])